### Mnist分类任务：

- 网络基本构建与训练方法，常用函数解析

- torch.nn.functional模块

- nn.Module模块


### 读取Mnist数据集
- 会自动进行下载

In [2]:
import torch
print(torch.__version__)

2.0.0


### 下载数据集

In [ ]:
%matplotlib inline

In [5]:
from pathlib import Path
import requests

DATA_PATH = Path("data")
PATH = DATA_PATH / "mnist"

PATH.mkdir(parents=True, exist_ok=True)

URL = "http://deeplearning.net/data/mnist/"
FILENAME = "mnist.pkl.gz"

if not (PATH / FILENAME).exists():
        content = requests.get(URL + FILENAME).content
        (PATH / FILENAME).open("wb").write(content)

### 读取

In [6]:
import pickle
import gzip

with gzip.open((PATH / FILENAME).as_posix(), "rb") as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding="latin-1")

In [8]:
x_train.shape

(50000, 784)

784是mnist数据集每个样本的像素点个数

In [9]:
from matplotlib import pyplot
import numpy as np

pyplot.imshow(x_train[0].reshape((28, 28)), cmap="gray")
print(x_train.shape)

ModuleNotFoundError: No module named 'matplotlib'

<img src="./img/4.png" alt="FAO" width="790">

<img src="./img/5.png" alt="FAO" width="790">

注意数据需转换成tensor才能参与后续建模训练


In [10]:
import torch

x_train, y_train, x_valid, y_valid = map(
    torch.tensor, (x_train, y_train, x_valid, y_valid)
)
n, c = x_train.shape
x_train, x_train.shape, y_train.min(), y_train.max()
print(x_train, y_train)
print(x_train.shape)
print(y_train.min(), y_train.max())

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]) tensor([5, 0, 4,  ..., 8, 4, 8])
torch.Size([50000, 784])
tensor(0) tensor(9)


### torch.nn.functional 很多层和函数在这里都会见到

torch.nn.functional中有很多功能，后续会常用的。那什么时候使用nn.Module，什么时候使用nn.functional呢？一般情况下，如果模型有可学习的参数，最好用nn.Module，其他情况nn.functional相对更简单一些

In [11]:
import torch.nn.functional as F

loss_func = F.cross_entropy

def model(xb):
    return xb.mm(weights) + bias

In [12]:
bs = 64
xb = x_train[0:bs]  # a mini-batch from x
yb = y_train[0:bs]
weights = torch.randn([784, 10], dtype = torch.float,  requires_grad = True) 
bs = 64
bias = torch.zeros(10, requires_grad=True)

print(loss_func(model(xb), yb))

tensor(11.3860, grad_fn=<NllLossBackward0>)


### 创建一个model来更简化代码

- 必须继承nn.Module且在其构造函数中需调用nn.Module的构造函数
- 无需写反向传播函数，nn.Module能够利用autograd自动实现反向传播
- Module中的可学习参数可以通过named_parameters()或者parameters()返回迭代器

In [13]:
from torch import nn

class Mnist_NN(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden1 = nn.Linear(784, 128)
        self.hidden2 = nn.Linear(128, 256)
        self.out  = nn.Linear(256, 10)

    def forward(self, x):
        x = F.relu(self.hidden1(x))
        x = F.relu(self.hidden2(x))
        x = self.out(x)
        return x
        

In [14]:
net = Mnist_NN()
print(net)


Mnist_NN(
  (hidden1): Linear(in_features=784, out_features=128, bias=True)
  (hidden2): Linear(in_features=128, out_features=256, bias=True)
  (out): Linear(in_features=256, out_features=10, bias=True)
)


可以打印我们定义好名字里的权重和偏置项

In [15]:
for name, parameter in net.named_parameters():
    print(name, parameter,parameter.size())

hidden1.weight Parameter containing:
tensor([[-0.0190,  0.0228, -0.0052,  ..., -0.0067, -0.0055,  0.0230],
        [-0.0126,  0.0175,  0.0004,  ...,  0.0117,  0.0303,  0.0109],
        [-0.0051,  0.0195, -0.0265,  ..., -0.0246, -0.0312, -0.0109],
        ...,
        [-0.0130, -0.0102, -0.0078,  ...,  0.0030, -0.0070,  0.0161],
        [ 0.0200,  0.0230, -0.0343,  ...,  0.0211, -0.0285, -0.0206],
        [ 0.0084, -0.0259,  0.0319,  ...,  0.0224,  0.0143, -0.0041]],
       requires_grad=True) torch.Size([128, 784])
hidden1.bias Parameter containing:
tensor([ 0.0347,  0.0294, -0.0321, -0.0212, -0.0046, -0.0290, -0.0307,  0.0214,
        -0.0210,  0.0118,  0.0045,  0.0072,  0.0038, -0.0352, -0.0357,  0.0289,
        -0.0264,  0.0161,  0.0357,  0.0077,  0.0311, -0.0242, -0.0132, -0.0026,
         0.0224,  0.0055, -0.0234, -0.0099, -0.0278, -0.0269, -0.0340, -0.0074,
         0.0174, -0.0160,  0.0008, -0.0178,  0.0242, -0.0090, -0.0209,  0.0178,
         0.0023,  0.0225, -0.0086,  0.0132, 

### 使用TensorDataset和DataLoader来简化

In [16]:
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

train_ds = TensorDataset(x_train, y_train)
train_dl = DataLoader(train_ds, batch_size=bs, shuffle=True)

valid_ds = TensorDataset(x_valid, y_valid)
valid_dl = DataLoader(valid_ds, batch_size=bs * 2)

In [17]:
def get_data(train_ds, valid_ds, bs):
    return (
        DataLoader(train_ds, batch_size=bs, shuffle=True),
        DataLoader(valid_ds, batch_size=bs * 2),
    )

- 一般在训练模型时加上model.train()，这样会正常使用Batch Normalization和 Dropout
- 测试的时候一般选择model.eval()，这样就不会使用Batch Normalization和 Dropout

In [18]:
import numpy as np

def fit(steps, model, loss_func, opt, train_dl, valid_dl):
    for step in range(steps):
        model.train()
        for xb, yb in train_dl:
            loss_batch(model, loss_func, xb, yb, opt)

        model.eval()
        with torch.no_grad():
            losses, nums = zip(
                *[loss_batch(model, loss_func, xb, yb) for xb, yb in valid_dl]
            )
        val_loss = np.sum(np.multiply(losses, nums)) / np.sum(nums)
        print('当前step:'+str(step), '验证集损失：'+str(val_loss))

In [19]:
from torch import optim
def get_model():
    model = Mnist_NN()
    return model, optim.SGD(model.parameters(), lr=0.001)

In [20]:
def loss_batch(model, loss_func, xb, yb, opt=None):
    loss = loss_func(model(xb), yb)

    if opt is not None:
        loss.backward()
        opt.step()
        opt.zero_grad()

    return loss.item(), len(xb)

### 三行搞定！

In [21]:
train_dl, valid_dl = get_data(train_ds, valid_ds, bs)
model, opt = get_model()
fit(25, model, loss_func, opt, train_dl, valid_dl)

当前step:0 验证集损失：2.2741712436676025
当前step:1 验证集损失：2.238810009384155
当前step:2 验证集损失：2.182846549987793
当前step:3 验证集损失：2.0916344299316405
当前step:4 验证集损失：1.9448151817321777
当前step:5 验证集损失：1.7322513929367065
当前step:6 验证集损失：1.4767965171813966
当前step:7 验证集损失：1.2307495523452758
当前step:8 验证集损失：1.0307839628219604
当前step:9 验证集损失：0.8800791104316712
当前step:10 验证集损失：0.7683674968719483
当前step:11 验证集损失：0.6856172645568848
当前step:12 验证集损失：0.6226232534408569
当前step:13 验证集损失：0.5744712522506714
当前step:14 验证集损失：0.536667770767212
当前step:15 验证集损失：0.5062088196277619
当前step:16 验证集损失：0.48142317361831666
当前step:17 验证集损失：0.4611051800251007
当前step:18 验证集损失：0.44392883687019347
当前step:19 验证集损失：0.4294238112449646
当前step:20 验证集损失：0.41689098663330076
当前step:21 验证集损失：0.4065068813085556
当前step:22 验证集损失：0.3965473276376724
当前step:23 验证集损失：0.38808579473495486
当前step:24 验证集损失：0.3806427438974381
